# era5land download for test years


In [ ]:
%%time
import cdsapi

# Replace 'YOUR_CDS_API_KEY' with your actual API key
api_key = '123057:e2f4481e-d1a8-412c-a1b4-fab7dd9b3931'

# Create a CDS API client
c = cdsapi.Client(key=api_key, url="https://cds.climate.copernicus.eu/api/v2")


# Define the variables and other parameters
variables = ["evaporation_from_bare_soil", "evaporation_from_vegetation_transpiration", "soil_temperature_level_1", "soil_temperature_level_2"]

years = [2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019, 2021, 2023]
area = [90, -180, -90, 180]  # North, West, South, East bounding box coordinates
hours = ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00",
         "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"]

# Loop over each variable
for variable in variables:
    # Loop over each year
    for year in years:
        # Define the request parameters with loops inside
        request_params = {
            "product_type": "reanalysis",
            "format": "netcdf",
            "variable": [variable],
            "year": [str(year)],
            "month": [f"{month:02d}" for month in range(1, 13)],
            "day": [f"{day:02d}" for day in range(1, 32)],
            "area": area,
            "time": hours,
        }

        # Make the request and download the data
        output_file = f"era5_land_{variable}_{year}.nc"
        c.retrieve("reanalysis-era5-land", request_params, output_file)

# Close the CDS API client
c.close()



# making time series per station for all era5 variables

In [ ]:
# Что нужно сделать:
# По ERA5-land для 26 постов нужно посчитать среднее значение параметра по водосборам. 
# Всего параметров 23 из списка ниже, отмечено зеленым. 
# Период расчета – с 1990 года по настоящее время. 
# dНа выходе нужны осредненные по водосборам суточные ряды по каждому параметру
# т.е. таблица вида «site_id - date – mean_value». 

# read shp files 
path = '/Users/varyabazilova/Desktop/drivendata_rodeo/data/'
gdf = gpd.read_file(path + 'geospatial.gpkg', layer='basins')

In [ ]:
# imports 

import xarray as xr
import rioxarray as rio
import pandas as pd
import numpy as np 
import geopandas as gpd
import matplotlib.pyplot as plt
import os

# correct longitudes
# test['longitude'] = np.arange(-122.4, -104.7 + 0.1, 0.1)


# ---------------------------------------------------
# ------- functions -----------------
# ---------------------------------------------------

# stat by shape for the variables that are !stored normally!

def stat_by_shape(xr_file, shape, kind='mean'):
    '''calculate zonal statistics for each polygon
    - write the "right" coordinates to the .nc file
    - write the same-as-polygons crs
    - clip the .nc file with the polygon bounds
    - resample hourly data to daily, remove useles variables
    - compute statistics
    - convert to pandas df, return df
    '''
    xr_file['longitude'] = np.arange(-122.4, -104.7 + 0.1, 0.1)
    xr_file = xr_file.rio.write_crs(shape.crs)
    xr_file = xr_file.rio.clip(shape.geometry)
    daily_mean = xr_file.resample(time='D').mean().drop_vars('spatial_ref')
    
    if kind=='mean':
        sr_pcpn = daily_mean.mean(dim=['latitude', 'longitude']).to_dataframe()
    if kind=='min':
        sr_pcpn = daily_mean.min(dim=['latitude', 'longitude']).to_dataframe()
    if kind=='max':
        sr_pcpn = daily_mean.max(dim=['latitude', 'longitude']).to_dataframe()
    if kind=='std':
        sr_pcpn = daily_mean.std(dim=['latitude', 'longitude']).to_dataframe()
    return sr_pcpn



# -------- stat by shape for the variables that are !stored cumulatively! for each forecasting period 
# (e.g. taking MIN per day for the evaporation)

def stat_by_shape_cumul(xr_file, shape, kind='mean'):
    '''same as 'stat by shape', 
    but to resample to dayly - take MIN 
    (total value per day)
    '''
    xr_file['longitude'] = np.arange(-122.4, -104.7 + 0.1, 0.1)
    xr_file = xr_file.rio.write_crs(shape.crs)
    xr_file = xr_file.rio.clip(shape.geometry)
    daily_mean = xr_file.resample(time='D').min().drop_vars('spatial_ref') # !!!
    
    if kind=='mean':
        sr_pcpn = daily_mean.mean(dim=['latitude', 'longitude']).to_dataframe()
    if kind=='min':
        sr_pcpn = daily_mean.min(dim=['latitude', 'longitude']).to_dataframe()
    if kind=='max':
        sr_pcpn = daily_mean.max(dim=['latitude', 'longitude']).to_dataframe()
    if kind=='std':
        sr_pcpn = daily_mean.std(dim=['latitude', 'longitude']).to_dataframe()
    return sr_pcpn



# ----- open all .nc files in a folder, clean the 'expver' dimention

def open_and_combine_nc_files(folder_path):
    """
    open all .nc files togetehr
    required: folder_path (str)
    return: combined dataset with combined 'expver' dim
    """
    # open all data together
    dataset = xr.open_mfdataset(os.path.join(folder_path, '*.nc'))

    # list of keys (dimensions)
    keys_list = list(dataset.dims.keys())
    # check if 'expver' is there, if so - combine
    if 'expver' in keys_list:
        dataset = dataset.sel(expver=1).combine_first(dataset.sel(expver=5))

    return dataset



## calculations for NORMAL data

In [ ]:
%%time
# Iterate over the folders and files in the specified path
basepath = '/Users/varyabazilova/Desktop/drivendata_rodeo/data/era5_land/'
# basepath = '/Users/varyabazilova/Desktop/drivendata_rodeo/data/era5test/'

folders = [d for d in os.listdir(basepath) if not d.startswith('.') and os.path.isdir(os.path.join(basepath, d))]

for folder in folders:
    ''' do the calcuations for all folders
    every folder should consist the data with one variable'''
    # identify folder
    folder_path = os.path.join(basepath, folder)
    # open and combine data
    dataset = open_and_combine_nc_files(folder_path)
    
    print(dataset)
    
    #create a table 
    table = pd.DataFrame()
    # gdf = gdf[:2]
    
    # iterate over every polygon in a shap file
    for site_id in gdf.site_id:
        
        #take one polygon
        shape = gdf[gdf.site_id == site_id]  
        
        #calculate stats 
        df1 = stat_by_shape(dataset, shape, kind='mean').rename(columns=lambda x: x + '_mean')        
        df2 = stat_by_shape(dataset, shape, kind='min').rename(columns=lambda x: x + '_min')
        df3 = stat_by_shape(dataset, shape, kind='max').rename(columns=lambda x: x + '_max')
        df5 = stat_by_shape(dataset, shape, kind='std').rename(columns=lambda x: x + '_std')
        # concat all togetehr
        df = pd.concat([df1, df2, df3, df5], axis=1) 
        #add site-id of the polygon
        df['site_id'] = shape['site_id'].iloc[0]
        #append to the table
        table = table.append(df)

        # Save the table as a CSV file with the folder name
    csv_filename = os.path.join(path_out, f'{folder}_table.csv')
    table.to_csv(csv_filename, index=True) # important index=True!!! - this will save DATE 


# res inflow - hardcoded downloads for ALL years 

In [ ]:
#  ----- what is going on here: -------

# this is the resercoir inflow data, that is being pulled from rise api by BOR 
# the API request is hard coded through the web site: 
# the url is aquired through the time-series quiry 
# link: https://data.usbr.gov/time-series/search?id=10773.19550301-20231213&it=10773&lo=334
# manually search for the name of the reserciir (select "water")
# choose reservoir inflow, click time-series-queuery -> generate API request 
# NB! one request-link only has ONE page and is able to store only 2000values
# => you need to iterate over PAGES in the link name to get more data


# 'site_id' matched with the name of a reservoir name 

# 0              hungry_horse_reservoir_inflow     Hungry Horse 
# 2                    pueblo_reservoir_inflow     Pueblo
# 8                           boise_r_nr_boise     Lucky Peak
# 10              taylor_park_reservoir_inflow     Taylor Park
# 12                    ruedi_reservoir_inflow     Ruedi
# 13               fontenelle_reservoir_inflow     Fontenelle
# 15     san_joaquin_river_millerton_reservoir     Folsom
# 17                american_river_folsom_lake     Friant
# 23                   boysen_reservoir_inflow     Boysen
# 25                    owyhee_r_bl_owyhee_dam     Owyhee


In [18]:
import requests
import pandas as pd

# ----- functions ------

def process_inflow(json):
    '''this function converts json data to pd df,
    re-formats time the correct way
    add site_id to the df'''
    
    df = pd.json_normalize(json['data'])
    
    # df = df[['attributes.dateTime', 'attributes.result']]
    df[['date', 'time']] = df['attributes.dateTime'].str.split('T', expand=True)
    
    return df


# --------- getting the data drom the api ----

def fetch_data(url, num_pages, site_id):
    ''' this fetches the data from the api:
    '''
    reservoir_data = pd.DataFrame()

    for page_num in range(1, num_pages):
        # Construct the URL with the current page number
        current_url = f'{url}&page={page_num}'
        
        # Make the request and process the data
        r = requests.get(current_url).json()
        
        #process inflow 
        df = pd.json_normalize(r['data'])
        df = df[['attributes.dateTime', 'attributes.result']]
        df[['date', 'time']] = df['attributes.dateTime'].str.split('T', expand=True)
        
        # df = process_inflow(r)
        
        # Append the data to the main dataframe
        reservoir_data = reservoir_data.append(df)

        # print(f'Processing page {page_num}')

    # Add the site_id column
    reservoir_data['site_id'] = site_id

    return reservoir_data



In [11]:

# order: folsom, boysen, taylor park, friant 


url_folsom = 'https://data.usbr.gov/rise/api/result?itemsPerPage=2000&order%5BdateTime%5D=ASC&itemId=10773&dateTime%5Bafter%5D=19550301&dateTime%5Bstrictly_before%5D=20231212' #folsom
url_boysen = 'https://data.usbr.gov/rise/api/result?itemsPerPage=2000&order%5BdateTime%5D=ASC&itemId=197&dateTime%5Bafter%5D=19111001&dateTime%5Bstrictly_before%5D=20230915' #boysen
url_TP = 'https://data.usbr.gov/rise/api/result?itemsPerPage=2000&order%5BdateTime%5D=ASC&itemId=794&dateTime%5Bafter%5D=19621001&dateTime%5Bstrictly_before%5D=20231213' #taylor park
url_friant = 'https://data.usbr.gov/rise/api/result?itemsPerPage=2000&order%5BdateTime%5D=ASC&itemId=10806&dateTime%5Bafter%5D=19440222&dateTime%5Bstrictly_before%5D=20231212' #friant

url_list= [url_folsom, url_boysen, url_TP, url_friant]
num_pages_list = [13, 22, 12, 16]
site_id_list = ['san_joaquin_river_millerton_reservoir', 'boysen_reservoir_inflow', 'taylor_park_reservoir_inflow', 'american_river_folsom_lake']


master_table = pd.DataFrame()

# Loop over the lists
for url, num_pages, site_id in zip(url_list, num_pages_list, site_id_list):
    # Call the fetch_data function
    data = fetch_data(url, num_pages, site_id)
    data = data[['attributes.result', 'date', 'site_id']]
    master_table = master_table.append(data)


Processing page 1
Processing page 2
Processing page 3
Processing page 4
Processing page 5
Processing page 6
Processing page 7
Processing page 8
Processing page 9
Processing page 10
Processing page 11
Processing page 12


In [16]:
master_table=master_table.rename(columns={'attributes.result': 'res_inflow'})#, 'B': 'Y'}, inplace=True)
master_table = master_table.drop_duplicates()

# make only test years 

In [17]:
master_table['date'] = pd.to_datetime(master_table['date'])
master_table['year'] = master_table['date'].dt.year

test_years =  [2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019, 2021, 2023]

master_table_test = master_table[master_table['year'].isin(test_years)]
